<a href="https://colab.research.google.com/github/virajpai/llm_experiments/blob/main/Document_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# installing libraries
!pip install 'transformers[torch]'
!pip install PyPDF2
!pip install opencv-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.2 MB/s eta 0:00:00


In [2]:
!pwd

/content


In [3]:
# import required libraries
from transformers import pipeline
import PyPDF2
import cv2
import PIL.Image as Image
import io
from os import walk

import numpy as np
import pandas as pd
from functools import reduce

from google.colab.patches import cv2_imshow


In [4]:
# Download text and image ero-shot-classifiers from hugginface
text_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
image_classifier = pipeline("zero-shot-image-classification", model="openai/clip-vit-large-patch14")


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [7]:
# labels for zero-short-tagging
labels = ["KYC documents", "Loan Application", "Bank Statement", "PAN Card", "Aadhaar Card"]

In [8]:
# define text and image scorer
def get_text_scores(doc_text, labels=labels):

  doc_text = doc_text.strip()

  if len(doc_text) > 25:
    text_result = text_classifier(doc_text, candidate_labels=labels)
    txt_df = pd.DataFrame([dict(zip(text_result['labels'], text_result['scores']))]).T.reset_index()
  else:
    txt_df = pd.DataFrame([dict(zip(labels, [1 / len(labels)] * len(labels)))]).T.reset_index()

  txt_df.columns = ['label', 'score']

  # display(txt_df)

  return txt_df


def get_img_scores(img, labels=labels):
  image_results = image_classifier(img, candidate_labels=labels)
  img_df = pd.DataFrame(image_results)

  # display(img_df)

  return img_df


In [9]:
# read file and get final scores

def pdf_score(pdf_file, labels=labels):
# get all image scores and finally cumulative text score

  doc_scores = []
  doc_text = ""
  pdf_reader = PyPDF2.PdfReader(pdf_file)

  for page in pdf_reader.pages:
    doc_text += page.extract_text()
    img_files = page.images

    if img_files and len(img_files) > 0:
      for im_f in img_files:
        img = Image.open(io.BytesIO(im_f.data))
        doc_scores.append(get_img_scores(img))

  doc_text_scores = get_text_scores(doc_text)

  if doc_text_scores['score'].sum() > 0:
    doc_scores.append(doc_text_scores)

  # build final dataframe with all scores merged by label
  df_final = reduce(lambda  left,right: pd.merge(left,right,on=['label'],
                                              how='inner'), doc_scores)

  # display(df_final)

  # final score = avg of all scores
  score_cols = [col for col in df_final.columns if 'score' in col]
  df_final['final_proba'] = df_final[score_cols].sum(axis=1) / len(score_cols)

  # extract the final predictions
  return df_final[['label', 'final_proba']].sort_values(
      by='final_proba', ascending=False).reset_index(
          drop=True).iloc[0].to_dict()

In [10]:
# pdf_file = open('documents/bank_stm.pdf', 'rb')

In [11]:
# pdf_score(pdf_file)

In [12]:
filenames = next(walk('./documents'), (None, None, []))[2]  # [] if no file

In [13]:
%%time
for f in filenames:
  pdf_file = './documents/' + f
  scores = pdf_score(pdf_file)
  print(f'File: {f}, Lable & Score: {scores}')
  print()

File: bank_stm.pdf, Lable & Score: {'label': 'Bank Statement', 'final_proba': 0.5970002770423889}

File: loan_app.pdf, Lable & Score: {'label': 'Loan Application', 'final_proba': 0.2909754812717438}

File: pan.pdf, Lable & Score: {'label': 'PAN Card', 'final_proba': 0.5788190424442291}

File: kyc_form.pdf, Lable & Score: {'label': 'Loan Application', 'final_proba': 0.49862966537475584}

CPU times: user 1min 6s, sys: 403 µs, total: 1min 6s
Wall time: 1min 7s
